## Setup

1. Install the `fastapi` package - a dependency for `glmpy`.

In [ ]:
!pip install fastapi==0.88.0

2. Install `glmpy`. This command looks a bit messy right now - currently it's hosted on a repository for testing packages.

In [ ]:
!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple glm-py --use-deprecated=legacy-resolver

## Run an existing `.nml` sim

This example assumes you already have a model configuration file (`.nml`) and any additional boundary condition files.

1. Import the `simulation` module

In [4]:
import glmpy.simulation as sim

2. Create a dictionary object of the names and paths for all input files, i.e., `{"my_file": "path/to/my_file"}`.

In [3]:
files = {
    "glm3.nml": "glm3.nml",
    "nldas_driver.csv": "bcs/nldas_driver.csv"
}

3. Initialise the `GlmSim` class by providing the file dictionary and specifying where the prepared model inputs should go (`inputs_dir`). Set `api=False` to run the simulation locally.

In [5]:
glm_sim = sim.GlmSim(
    input_files=files,
    api=False,
    inputs_dir="inputs"
)

4. Call the `prepare_inputs()` method to organise the input files inside the `"inputs"` directory in a way that GLM expects.

In [6]:
inputs_dir = glm_sim.prepare_inputs()

In the shell, list all files/subdirectories within your current working directory with the `ls` command (use `!` to run shell commands in python notebooks). You will see the newly created `inputs` subdirectory.

In [8]:
!ls

bcs  glm3.nml  glm3_from_json.nml  glmpy_demo.ipynb  inputs  sparkling-nml.json


Now change your working directory (`cd`) to the `inputs` subdirectory and list its contents - the `glm3.nml` file and a `bcs` subdirectory of boundary condition files.

In [9]:
!cd inputs
!ls inputs

bcs  glm3.nml


5. Run the simulation by calling the `glm_run()` method on the `glm_sim` object.

In [10]:
glm_sim.glm_run(
    inputs_dir=inputs_dir
)

/usr/local/lib/python3.10/site-packages/glmpy/bin/glm


Cannot open default display

     Reading configuration from glm3.nml
     VolAtCrest= 5830612.91654; MaxVol= 5830612.91654 (m3)
     No 'snowice' section, setting defaults & assuming no snowfall
Directory "output" does not exist - attempting to create it


     
    -------------------------------------------------------
    |  General Lake Model (GLM)   Version 3.3.1a11        |
    -------------------------------------------------------
     
     glm built using gcc version 10.2.1
     build date 20231208-0414UTC

     nDays= 730; timestep= 3600.000000 (s)
     Maximum lake depth is 18.288000
     Depth where flow will occur over the crest is 18.288000

     Wall clock start time :  Thu Jan 25 05:27:33 2024

     Simulation begins...
     Running day  2444345, 0.14% of days complete
     Running day  2444346, 0.27% of days complete
     Running day  2444347, 0.41% of days complete
     Running day  2444348, 0.55% of days complete
     Running day  2444349, 0.68% of days complete
     Running day  2444350, 0.82% of days complete
     Running day  2444351, 0.96% of days complete
     Running day  2444352, 1.09% of days complete
     Running day  2444353, 1.23% of days complete
     Running day  2444354, 1.37% of days complete
     Runni

## Create your own simulation

This example guides you through setting your own model parameters using the `nml` module. We'll recreate the Sparkling Lake simulation.

1. Import the `nml` and `json` modules. We'll use the latter one to speed things up later.

In [11]:
from glmpy import nml, json

Recall that a `.nml` file looks something like this:

```
&glm_setup
  sim_name = 'GLMSimulation'
  ...
/
&morphometry
  lake_name = 'my_lake'
  ...
/
&time
  timefmt = 3
  ...
/
&init_profiles
  lake_depth = 10
  ...
/
```

Each 'block' configures a specific aspect of the simulation, e.g., the lake morphometry with the `&morphometry` block. The `nml` module provides a class for each block, e.g., the `NMLMorphometry` class. Parameters are set by passing values to the respective argument of the class.

2. Start by configuring the `&glm_setup` block with the `NMLSetup` class.

In [31]:
setup = nml.NMLSetup(
    sim_name='Sparkling Lake',
    max_layers=500,
    min_layer_vol=0.5,
    min_layer_thick=0.15,
    max_layer_thick=0.5,
    density_model=1,
    non_avg=True
)

You can initalise the class by setting as many, or as few, arguments as you like. These can up set/updated later with a dictionary object and the `set_attributes()` method.

In [32]:
setup = nml.NMLSetup()

setup_attrs = {
    'sim_name': 'Sparkling Lake',
    'max_layers': 500,
    'min_layer_vol': 0.5,
    'min_layer_thick': 0.15,
    'max_layer_thick': 0.5,
    'density_model': 1,
    'non_avg': True
}

setup.set_attributes(setup_attrs)

3. Inspect the formatted result by printing the setup object. 

In [14]:
print(setup)

&glm_setup 
   sim_name = 'Sparkling Lake'
   max_layers = 500
   min_layer_vol = 0.5
   min_layer_thick = 0.15
   max_layer_thick = 0.5
   density_model = 1
   non_avg = .true.
/


4. Let's repeat this for the `&glm_mixing` block. Call the respective class in the `nml` module and set the attributes.

In [33]:
mixing = nml.NMLMixing(
    surface_mixing=1,
    coef_mix_conv=0.2,
    coef_wind_stir=0.402,
    coef_mix_shear=0.2,
    coef_mix_turb=0.51,
    coef_mix_KH=0.3,
    deep_mixing=2,
    coef_mix_hyp=0.5,
    diff=0.0
)

In [16]:
print(mixing)

&mixing 
   surface_mixing = 1
   coef_mix_conv = 0.2
   coef_wind_stir = 0.402
   coef_mix_shear = 0.2
   coef_mix_turb = 0.51
   coef_mix_KH = 0.3
   deep_mixing = 2
   coef_mix_hyp = 0.5
   diff = 0.0
/


Rebuilding the whole Sparkling Lake simulation will take some time. If you feel confident with the process so far, let's set the remaining blocks by retrieving the model parameters from a JSON file - a format commonly used to store data for use with web applications and servers.

5. Using the `json` module we imported earlier read in the `sparkling_lake.json` file using the `JSONToNML` class.

In [34]:
json_nml = json.JSONToNML("sparkling-nml.json")

6. We can now return a dictionary object of the model parameters for each nml block. 

In [35]:
morphometry_attrs = json_nml.get_nml_attributes("&morphometry")

In [26]:
print(morphometry_attrs)

{'lake_name': 'Sparkling', 'latitude': 46.00881, 'longitude': -89.69953, 'crest_elev': 320.0, 'bsn_len': 901.0385, 'bsn_wid': 901.0385, 'bsn_vals': 15, 'H': [301.712, 303.018285714286, 304.324571428571, 305.630857142857, 306.937142857143, 308.243428571429, 309.549714285714, 310.856, 312.162285714286, 313.468571428571, 314.774857142857, 316.081142857143, 317.387428571429, 318.693714285714, 320, 321], 'A': [0, 45545.8263571429, 91091.6527142857, 136637.479071429, 182183.305428571, 227729.131785714, 273274.958142857, 318820.7845, 364366.610857143, 409912.437214286, 455458.263571429, 501004.089928571, 546549.916285714, 592095.742642857, 637641.569, 687641.569]}


7. Let's use this dictionary to set the attributes of the `NMLMorphometry` class.

In [36]:
morphometry = nml.NMLMorphometry()
morphometry.set_attributes(morphometry_attrs)
print(morphometry)

&morphometry 
   lake_name = 'Sparkling'
   latitude = 46.00881
   longitude = -89.69953
   crest_elev = 320.0
   bsn_len = 901.0385
   bsn_wid = 901.0385
   bsn_vals = 15
   H = 301.712, 303.018285714286, 304.324571428571, 305.630857142857, 306.937142857143, 308.243428571429, 309.549714285714, 310.856, 312.162285714286, 313.468571428571, 314.774857142857, 316.081142857143, 317.387428571429, 318.693714285714, 320, 321
   A = 0, 45545.8263571429, 91091.6527142857, 136637.479071429, 182183.305428571, 227729.131785714, 273274.958142857, 318820.7845, 364366.610857143, 409912.437214286, 455458.263571429, 501004.089928571, 546549.916285714, 592095.742642857, 637641.569, 687641.569
/


8. Repeat for the remaining blocks

In [37]:
time_attrs=json_nml.get_nml_attributes("&time")
wq_setup_attrs=json_nml.get_nml_attributes("&wq_setup")
output_attrs=json_nml.get_nml_attributes("&output")
init_profiles_attrs=json_nml.get_nml_attributes("&init_profiles")
meteorology_attrs = json_nml.get_nml_attributes("&meteorology")
bird_model_attrs=json_nml.get_nml_attributes("&bird_model")
light_attrs=json_nml.get_nml_attributes("&light")
sediment_attrs=json_nml.get_nml_attributes("&sediment")

In [38]:
time = nml.NMLTime()
wq_setup = nml.NMLWQSetup()
output = nml.NMLOutput()
init_profiles = nml.NMLInitProfiles()
meteorology = nml.NMLMeteorology()
bird_model = nml.NMLBirdModel()
light = nml.NMLLight()
sediment = nml.NMLSediment()

In [39]:
time.set_attributes(time_attrs)
wq_setup.set_attributes(wq_setup_attrs)
output.set_attributes(output_attrs)
init_profiles.set_attributes(init_profiles_attrs)
meteorology.set_attributes(meteorology_attrs)
bird_model.set_attributes(bird_model_attrs)
light.set_attributes(light_attrs)
sediment.set_attributes(sediment_attrs)

9. Now that we have the attributes set for each block, the `.nml` file can be create and saved to file. First, create an instance of the `NML` class and pass in the configured blocks. Using the `write_nml()` method, the .nml can be saved to your directory.

In [40]:
nml = nml.NML(
  setup=setup,
  mixing=mixing,
  morphometry=morphometry,
  time=time,
  output=output,
  init_profiles=init_profiles,
  meteorology=meteorology,
  bird_model=bird_model,
  light=light,
  sediment=sediment
)

10. Save the file with the `write_nml()` method.

In [ ]:
nml.write_nml(nml_file_path='glm3_from_json.nml')

11. Finally, repeat the steps you did in the first example to prepare the inputs and run GLM

In [ ]:
files = {
    "glm3.nml": "glm3_from_json.nml",
    "nldas_driver.csv": "bcs/nldas_driver.csv"
}

glm_sim = sim.GlmSim(
    input_files=files,
    api=False,
    inputs_dir="inputs"
)

inputs_dir = glm_sim.prepare_inputs()

glm_sim.glm_run(
    inputs_dir=inputs_dir
)

## Run a sim from web-app `.json` inputs

Assumes a web-app has created a `.json` file of GLM NML parameters.

1. Import the `json` and `nml` modules

In [18]:
from glmpy import json, nml

2. Initialise the `JSONToNML` class by providing the file path to the JSON file.

In [19]:
json_nml = json.JSONToNML("sparkling-nml.json")

3. Fetch the `&glm_setup` parameters from the JSON file to use as attributes for the `NMLSetup` class. A dictionary of key/value pairs is returned.

In [24]:
setup_attrs = json_nml.get_nml_attributes("&glm_setup")
print(setup_attrs)

{'sim_name': 'Sparkling Lake', 'max_layers': 500, 'min_layer_vol': 0.5, 'min_layer_thick': 0.15, 'max_layer_thick': 0.5, 'density_model': 1, 'non_avg': True}


4. Initialise the `NMLSetup` class and set it's attributes from `setup_attrs` dictionary with the `set_attributes()` method.

In [25]:
setup = nml.NMLSetup()
setup.set_attributes(setup_attrs)
print(setup)

&glm_setup 
   sim_name = 'Sparkling Lake'
   max_layers = 500
   min_layer_vol = 0.5
   min_layer_thick = 0.15
   max_layer_thick = 0.5
   density_model = 1
   non_avg = .true.
/


5. Repeat for the remaining NML blocks

In [28]:
mixing_attrs = json_nml.get_nml_attributes("&mixing")
morphometry_attrs = json_nml.get_nml_attributes("&morphometry")
time_attrs=json_nml.get_nml_attributes("&time")
wq_setup_attrs=json_nml.get_nml_attributes("&wq_setup")
output_attrs=json_nml.get_nml_attributes("&output")
init_profiles_attrs=json_nml.get_nml_attributes("&init_profiles")
meteorology_attrs = json_nml.get_nml_attributes("&meteorology")
bird_model_attrs=json_nml.get_nml_attributes("&bird_model")
light_attrs=json_nml.get_nml_attributes("&light")
sediment_attrs=json_nml.get_nml_attributes("&sediment")

In [29]:
mixing = nml.NMLMixing()
morphometry = nml.NMLMorphometry()
time = nml.NMLTime()
wq_setup = nml.NMLWQSetup()
output = nml.NMLOutput()
init_profiles = nml.NMLInitProfiles()
meteorology = nml.NMLMeteorology()
bird_model = nml.NMLBirdModel()
light = nml.NMLLight()
sediment = nml.NMLSediment()

In [30]:
mixing.set_attributes(mixing_attrs)
morphometry.set_attributes(morphometry_attrs)
time.set_attributes(time_attrs)
wq_setup.set_attributes(wq_setup_attrs)
output.set_attributes(output_attrs)
init_profiles.set_attributes(init_profiles_attrs)
meteorology.set_attributes(meteorology_attrs)
bird_model.set_attributes(bird_model_attrs)
light.set_attributes(light_attrs)
sediment.set_attributes(sediment_attrs)

6. Initialise the `NML` class and pass in the `nml` block classes

In [31]:
nml = nml.NML(
  setup=setup,
  mixing=mixing,
  morphometry=morphometry,
  time=time,
  output=output,
  init_profiles=init_profiles,
  meteorology=meteorology,
  bird_model=bird_model,
  light=light,
  sediment=sediment
)

7. Write the NML to file with the `write_nml()` method

In [ ]:
nml.write_nml(nml_file_path='glm3_from_json.nml')

8. Setup and run the simulation

In [35]:
files = {
    "glm3.nml": "glm3_from_json.nml",
    "nldas_driver.csv": "bcs/nldas_driver.csv"
}

In [36]:
glm_sim = sim.GlmSim(
    input_files=files,
    api=False,
    inputs_dir="inputs"
)

In [37]:
inputs_dir = glm_sim.prepare_inputs()

In [38]:
glm_sim.glm_run(
    inputs_dir=inputs_dir
)

/usr/local/lib/python3.10/site-packages/glmpy/bin/glm
     
    -------------------------------------------------------
    |  General Lake Model (GLM)   Version 3.3.1a11        |
    -------------------------------------------------------
     
     glm built using gcc version 10.2.1
     build date 20231208-0414UTC

     nDays= 730; timestep= 3600.000000 (s)
     Maximum lake depth is 18.288000
     Depth where flow will occur over the crest is 18.288000

     Wall clock start time :  Fri Dec  8 07:08:17 2023

     Simulation begins...
     Running day  2444345, 0.14% of days complete
     Running day  2444346, 0.27% of days complete
     Running day  2444347, 0.41% of days complete
     Running day  2444348, 0.55% of days complete
     Running day  2444349, 0.68% of days complete
     Running day  2444350, 0.82% of days complete
     Running day  2444351, 0.96% of days complete
     Running day  2444352, 1.09% of days complete
     Running day  2444353, 1.23% of days complete
     R

Cannot open default display

     Reading configuration from glm3.nml
     VolAtCrest= 5830612.91654; MaxVol= 5830612.91654 (m3)
     No 'snowice' section, setting defaults & assuming no snowfall
     No 'inflow' config, assuming no inflows
     No 'outflow' config, assuming no outflows
Directory "output" does not exist - attempting to create it


     Running day  2444525, 24.76% of days complete
     Running day  2444526, 24.90% of days complete
     Running day  2444527, 25.03% of days complete
     Running day  2444528, 25.17% of days complete
     Running day  2444529, 25.31% of days complete
     Running day  2444530, 25.44% of days complete
     Running day  2444531, 25.58% of days complete
     Running day  2444532, 25.72% of days complete
     Running day  2444533, 25.85% of days complete
     Running day  2444534, 25.99% of days complete
     Running day  2444535, 26.13% of days complete
     Running day  2444536, 26.27% of days complete
     Running day  2444537, 26.40% of days complete
     Running day  2444538, 26.54% of days complete
     Running day  2444539, 26.68% of days complete
     Running day  2444540, 26.81% of days complete
     Running day  2444541, 26.95% of days complete
     Running day  2444542, 27.09% of days complete
     Running day  2444543, 27.22% of days complete
     Running day  2444544, 27.3

## Documentation

Read more about `glmpy` [here](https://wet-tool.github.io/glm-py/).